In [10]:
import sys

import numpy as np
import pandas as pd
import s3fs

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.helpers.io as io
import src.helpers.data as hd
import src.data.aggregators as agg
import src.data.selectors as sl

fs = s3fs.S3FileSystem(profile="3di")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fs.ls('s3://3di-data-mdb/clean')

['3di-data-mdb/clean/',
 '3di-data-mdb/clean/mdb_000.parquet',
 '3di-data-mdb/clean/mdb_111.parquet',
 '3di-data-mdb/clean/mdb_777.parquet',
 '3di-data-mdb/clean/mdb_X11.parquet',
 '3di-data-mdb/clean/mdb_X77.parquet',
 '3di-data-mdb/clean/mdb_costa.parquet',
 '3di-data-mdb/clean/pieces',
 '3di-data-mdb/clean/samples']

In [44]:
df = io.read_parquet('s3://3di-project-eval/eval_111.parquet')
hd.inspect(df)

shape: (0, 23), users: 0


,user_id,ym,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows_norm,t,month_spend,pct_credit,age,is_female,region_name,is_urban,has_sa_account,generation


False

## txn data dev

In [12]:
dfm = io.read_parquet('s3://3di-data-mdb/clean/mdb_111.parquet')

In [43]:
agg.sa_check(dfm).groupby('user_id').max()

Time for sa_check                      : 0.04 seconds


user_id
111        True
1111      False
5111      False
6111      False
7111       True
          ...  
583111    False
584111    False
587111     True
588111    False
589111     True
Name: has_sa_account, Length: 270, dtype: bool

In [ ]:
kk